In [1]:
import pandas as pd
import sqlite3

connection = sqlite3.connect('../data/checking-logs.sqlite')

In [2]:
pd.read_sql(sql="PRAGMA table_info(test);", con=connection)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [3]:
pd.read_sql(sql="SELECT * FROM test LIMIT 10;", con=connection)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [4]:
df_min=pd.read_sql(sql="""
            SELECT uid, MIN((strftime('%s', first_commit_ts) - deadlines) / 3600)  AS min FROM test AS t
            LEFT JOIN deadlines AS d ON d.labs=t.labname
            WHERE t.labname NOT LIKE "project1"
            """, con=connection)

In [5]:
df_min

,uid,min
0,user_30,-202


In [6]:
df_max=pd.read_sql(sql="""
            SELECT uid, MAX((strftime('%s', first_commit_ts) - deadlines) / 3600) AS max FROM test AS t
            LEFT JOIN deadlines AS d ON d.labs=t.labname
            WHERE t.labname NOT LIKE "project1"
            """, con=connection)

In [7]:
df_max

,uid,max
0,user_25,-2


In [8]:
df_avg=pd.read_sql(sql="""
            SELECT AVG((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg FROM test AS t
            LEFT JOIN deadlines AS d ON d.labs=t.labname
            WHERE t.labname NOT LIKE "project1" 
            """, con=connection)
df_avg
#unixepoch

,avg
0,-89.125


In [19]:
views_diff=pd.read_sql(sql="""
            SELECT t.uid, ((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff, COUNT(DISTINCT p.datetime) AS pageviews FROM test AS t
            LEFT JOIN deadlines AS d ON d.labs=t.labname
            LEFT JOIN pageviews AS p ON p.uid=t.uid
            WHERE t.labname NOT LIKE "project1"
            GROUP BY t.uid
            """, 
            con=connection)
views_diff

,uid,avg_diff,pageviews
0,user_1,-6,28
1,user_10,-39,89
2,user_14,-200,143
3,user_17,-81,47
4,user_18,-4,3
5,user_19,-148,16
6,user_21,-126,10
7,user_25,-148,179
8,user_28,-98,149
9,user_3,-75,317


In [20]:
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.062967
pageviews,-0.062967,1.000000


In [12]:
views_diff = pd.read_sql("""
    SELECT t.uid,
           t.avg_diff,
           p.views_count
    FROM (
        SELECT uid,
               ((strftime('%s', first_commit_ts) - deadlines) / 3600) AS avg_diff
        FROM test AS t
        LEFT JOIN deadlines AS d ON d.labs = t.labname
        WHERE t.labname NOT LIKE 'project1'
        GROUP BY uid
    ) t
    LEFT JOIN (
        SELECT uid,
               COUNT(*) AS views_count
        FROM pageviews
        GROUP BY uid
    ) p
    ON t.uid = p.uid
""", con=connection)

views_diff

,uid,avg_diff,views_count
0,user_1,-6,28
1,user_10,-39,89
2,user_14,-200,143
3,user_17,-81,47
4,user_18,-4,3
5,user_19,-148,16
6,user_21,-126,10
7,user_25,-148,179
8,user_28,-98,149
9,user_3,-75,317


In [22]:
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,-0.062967
pageviews,-0.062967,1.000000


In [ ]:
connection.close()